In [ ]:
import sys, os, argparse
import time
import netCDF4 as nc
from netCDF4 import Dataset
import numpy as np
from numpy import ma 

#Import packages for plotting
from matplotlib import pyplot as plt
from matplotlib import colors as mcolors
from pylab import imshow,cm
import cartopy.crs as ccrs 
import cartopy.feature as cfeature  

#Import packages for interpolating
from scipy.interpolate import griddata
from scipy.interpolate import LinearNDInterpolator as LNDI

# Import package for parallel computing
from joblib import Parallel, delayed


# Get the parent directory
parent_directory = "/cluster/home/llpui9007/Programs/HPC_Spectral_Clustering"
FTLE_parent_directory = "/cluster/home/llpui9007/Programs/FTLE"
TBarrier_parent_directory = "/cluster/home/llpui9007/Programs/TBarrier-main/TBarrier/2D"


# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/Similarity_matrix_clustering")
sys.path.append(parent_directory+"/subfunctions/Similarity_matrix_construction")
sys.path.append(parent_directory+"/subfunctions/trajectory_advection")
sys.path.append(parent_directory+"/subfunctions/Parallelisation")
sys.path.append(parent_directory+"/subfunctions/latlon_transform")
sys.path.append(parent_directory+"/utils")
sys.path.append(FTLE_parent_directory)
# add utils folder to current working path
sys.path.append(TBarrier_parent_directory+"/subfunctions/utils")
# add integration folder to current working path
sys.path.append(TBarrier_parent_directory+"/subfunctions/integration")
# add FTLE folder to current working path
sys.path.append(TBarrier_parent_directory+"/demos/AdvectiveBarriers/FTLE2D")


# Import linear interpolation function for unsteady flow field with irregular grid
from Interpolant import generate_mask_interpolator , generate_velocity_interpolants
# Import function to compute flow map/particle trajectories
from regular_regrid import regular_grid_interpolation_scalar
from polar_rotation import polar_rotation_rx 

#Parallelisation folder
from NetCDF_generator import generate_regrided

# Import function to compute gradient of flow map
from gradient_flowmap import gradient_flowmap
from gradient_flowmap_support import interpolant_unsteady
# Import function to compute finite time Lyapunov exponent (FTLE)
from FTLE import parallel_FTLE
from ploters import plotpolar_scatter_masked_ftle
from days_since_to_date import days_since_to_date


In [ ]:
def calculate_ftle_in_coordinates(x0,y0,reg_vel_file_path,dic,dt,tmin,tmax, NCores):
    # Read dataset
    print("Reading regrided input data")
    dataset = nc.Dataset(reg_vel_file_path, mode='r')
    #from m/s to m/day
    interpolated_siu = dataset.variables['vlon'][tmin:tmax,:,:]
    interpolated_siv = dataset.variables['vlat'][tmin:tmax,:,:]
    interpolated_siu = np.transpose(interpolated_siu, axes=(1, 2, 0))
    interpolated_siv = np.transpose(interpolated_siv, axes=(1, 2, 0))
    regrided_land_mask = dataset.variables['land_mask'][:,:]
    # Access coordinates
    lat_grid = dataset.variables['regrided_rot_lat'][:]  
    lon_grid = dataset.variables['regrided_rot_lon'][:]
    # Access specific variables
    time_data = dataset.variables['time'][tmin:tmax] 
    time_data= np.reshape(time_data, (1,-1))
    dataset.close()


    # Find the points where the velocity arrays are 0. This means either land or null initial velocity and therefore we don't 
    # want to have IC there.
    vel_land_interpolator = generate_mask_interpolator(lat_grid,lon_grid,interpolated_siu,interpolated_siv)
    print(interpolated_siu.shape)
    print("shape of interpolated siu")
    Interpolant_u, Interpolant_v = generate_velocity_interpolants(interpolated_siu, interpolated_siv,lon_grid, lat_grid, NCores)

    lat_grid = lat_grid.filled()
    lon_grid = lon_grid.filled()


    # Time resolution of data
    dt_data = time_data[0, 1]-time_data[0,0] # float

    # Periodic boundary conditions
    periodic_x = False # bool
    periodic_y = False # bool
    periodic_t = False # bool
    periodic = [periodic_x, periodic_y, periodic_t]

    # Unsteady velocity field
    bool_unsteady = True # bool

    # Defined domain
    defined_domain = np.isfinite(interpolated_siu[:,:,0]).astype(int) # array (NY, NX)

    ## Compute meshgrid of dataset
    X, Y =  lon_grid, lat_grid # array (NY, NX)

    ## Resolution of meshgrid
    dx_data = X[0,1]-X[0,0] # float
    dy_data = Y[1,0]-Y[0,0] # float

    delta = [dx_data, dy_data] # list (2, )


    # Initial time (in days)
    t0 = time_data[0,0] # float

    # Final time (in days)
    tN = time_data[0,-1] # float

    # NOTE: For computing the backward trajectories: tN < t0 and dt < 0.

    time = np.arange(t0, tN+dt, dt) # shape (Nt,)

    # Length of time interval (in days)
    lenT = abs(tN-t0) # float



    # Define ratio of auxiliary grid spacing vs original grid_spacing
    aux_grid = [aux_grid_ratio*dic, aux_grid_ratio*dic] # list (2, )


    # Split x0, y0 into 'Ncores' equal batches for parallel computing
    def split(a, n):
        k, m = divmod(len(a), n)
        return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

    x0_batch = list(split(x0, NCores)) # list (Nx*Ny)
    y0_batch = list(split(y0, NCores)) # list (Nx*Ny)


    # compute FTLE
    results = Parallel(n_jobs=Ncores, verbose = 50)(delayed(parallel_FTLE)(time, x0_batch[i], y0_batch[i], X, Y, Interpolant_u, Interpolant_v, periodic, defined_domain, bool_unsteady, time_data, aux_grid) for i in range(len(x0_batch)))

    # Extract FTLE from results of parallel computing
    FTLE = results[0]

    for res in results[1:]:
        FTLE = np.append(FTLE, res)
        
    # Reshape array from vectorized form to structured meshgrid
    FTLE = FTLE.reshape((X_domain.shape[0], X_domain.shape[1])) # array (Ny, Nx)

    #Land mask: water 0, land 1
    mask_interpolator = LNDI(list(zip(lat_grid.ravel(), lon_grid.ravel())), regrided_land_mask.ravel(),fill_value=1)
    ftle_land_mask=mask_interpolator(Y_domain,X_domain)
    masked_ftle = np.ma.masked_array(FTLE, mask=ftle_land_mask)
    return masked_ftle


